# Solving MNIST with GZIP and code-golfing into obscurity

In [1]:
import gzip
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.datasets import fetch_openml

In [2]:
# Fetch and split MNIST data
print("Fetching and splitting MNIST data...")
mnist = fetch_openml('mnist_784')
X, y = mnist.data.astype('float32'), mnist.target.astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Fetching and splitting MNIST data...


c:\Users\jakobserlier\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Creating a training set...
Creating a test set...


In [7]:
# Create a training set with 20 samples for each label
print("Creating a training set...")
training_set = []
for label in np.unique(y_train):
    label_data = X_train[y_train == label].head(100)
    samples = [(row.values, label) for _, row in label_data.iterrows()]
    training_set.extend(samples)

# Create a test set
print("Creating a test set...")
test_set_df = X_test.copy()
test_set_df['label'] = y_test
test_set = [(row.drop('label').values, row['label']) for index, row in test_set_df.iterrows()]

Creating a training set...
Creating a test set...


In [9]:
def compute_ncd(x1, x2):
    """Compute the Normalized Compression Distance (NCD) between two samples."""
    Cx1 = len(gzip.compress(x1.tobytes()))
    Cx2 = len(gzip.compress(x2.tobytes()))
    Cx1x2 = len(gzip.compress((x1 + x2).tobytes()))
    
    return (Cx1x2 - min(Cx1, Cx2)) / max(Cx1, Cx2)

print("Classifying test samples...")

k = 5  # Number of neighbors to consider
correct_predictions = 0  # Counter for correct predictions
actual_labels = []
predicted_labels = []

# Cache compressed lengths for training samples
compressed_lengths = [(x, len(gzip.compress(x.tobytes())), label) for x, label in training_set]

for (x1, actual_label) in tqdm(test_set[:100]):
    # Calculate NCD for each training sample
    distances = [(compute_ncd(x1, x), label) for x, _, label in compressed_lengths]
    
    # Get k nearest neighbors and predict label
    neighbors = sorted(distances, key=lambda x: x[0])[:k]
    top_k_class = [label for _, label in neighbors]
    predicted_class = Counter(top_k_class).most_common(1)[0][0]
    
    # Update predictions and counts
    actual_labels.append(actual_label)
    predicted_labels.append(predicted_class)
    correct_predictions += (predicted_class == actual_label)

# Print results and confusion matrix
print(f"Accuracy: {correct_predictions / len(test_set[:100]) * 100:.2f}%")
print("Generating confusion matrix...")
print(confusion_matrix(actual_labels, predicted_labels))


Classifying test samples...


100%|██████████| 100/100 [04:37<00:00,  2.78s/it]

Accuracy: 78.00%
Generating confusion matrix...
[[ 8  0  0  0  0  0  2  0  1  0]
 [ 0  9  0  0  0  0  0  0  0  0]
 [ 0  0  6  0  0  0  0  0  1  0]
 [ 1  0  0  8  0  0  0  0  2  0]
 [ 1  0  0  0  4  0  0  0  0  2]
 [ 1  0  0  0  0  6  0  0  2  0]
 [ 0  0  0  0  0  1 10  0  0  0]
 [ 0  0  0  0  0  0  0  8  2  4]
 [ 0  0  0  1  0  0  0  0  9  1]
 [ 0  0  0  0  0  0  0  0  0 10]]


# Code golfed

Code golfed into obscurity.

For a more accurate accuracy, use more samples.

In [22]:
ts = test_set[:100]

In [23]:
c = lambda z: len(gzip.compress(z.tobytes()))

def ncd(x, y):
    return (c(x + y) - min(c(x), c(y))) / max(c(x), c(y))

cls = [(x, c(x), l) for x, l in training_set]

correct_predictions = sum([np.array_equal(Counter([l for _, _, l in sorted([(ncd(x1, x), x, l) for x, _, l in cls], key=lambda t: t[0])[:5]]).most_common(1)[0][0], label) for x1, label in ts])

In [24]:
print("Accuracy:", correct_predictions / len(ts) * 100)

Accuracy: 78.0
